ここで用いられているソースコードはhttps://github.com/Wildqat/Wildqat/blob/master/examples_ja/tutorial011_knapsack_with_integer_weights.ipynb
を参考にして書いています。

$H _ { A }$  
$ \displaystyle
=A \left\{
    -2 \left( \sum _ { i = 1 } ^ { W_{limit} } y_i \right)
    +\left( \sum _ { i = 1 } ^ { W_{limit} } y_i \right) ^ { 2 }
    +\left( \sum _ { i = 1 } ^ { W_{limit} } iy_i \right) ^ { 2 }
    -2 \left( \sum _ { i = 1 } ^ { W_{limit} } iy_i \right) \left( \sum _ \alpha w_\alpha x_\alpha \right)
    +\left( \sum _ \alpha w_\alpha x_\alpha \right) ^ 2 \right\}
$

$H_A$
$
= \lambda \Big(1 - \sum_{n=1}^W y_n \Big)^2 + \mu \Big(\sum_{n=1}^W n y_n - \sum_{i=1}^N w_i x_i \Big)^2 
$
$
= \lambda \Big[1 - 2 \sum_{n=1}^W y_n +  \Big(\sum_{n=1}^W y_n\Big)^2 \Big] + \mu \Big[\Big(\sum_{n=1}^W ny_n\Big)^2 -2\Big(\sum_{n=1}^W ny_n\Big)\Big(\sum_{i=1}^N w_i x_i\Big) +  \Big(\sum_{i=1}^N w_i x_i\Big)^2 \Big]
$

H_A &=& \lambda \Big(1 - \sum_{n=1}^W y_n \Big)^2 + \mu \Big(\sum_{n=1}^W n y_n - \sum_{i=1}^N w_i x_i \Big)^2 \nonumber \\
&=& \lambda \Big[1 - 2 \sum_{n=1}^W y_n +  \Big(\sum_{n=1}^W y_n\Big)^2 \Big]  \nonumber \\
&& + \mu \Big[\Big(\sum_{n=1}^W ny_n\Big)^2 -2\Big(\sum_{n=1}^W ny_n\Big)\Big(\sum_{i=1}^N w_i x_i\Big) +  \Big(\sum_{i=1}^N w_i x_i\Big)^2 \Big] \nonumber \\
&=& \lambda \Big[1 - 2 \sum_{n=1}^W y_n + \Big(\sum_{n=1}^W y_n^2 + \sum_{1 \leq n < m \leq W} 2 y_n y_m \Big) \Big]  \nonumber \\
&& + \mu\Big[\Big(\sum_{n=1}^W n^2 y_n^2 + \sum_{1 \leq n < m \leq W} 2 n m y_n y_m\Big) \nonumber \\
&& -2\Big(\sum_{n=1}^W ny_n\Big)\Big(\sum_{i=1}^N w_i x_i\Big) +  \Big(\sum_{i=1}^N w_{i}^2 x_{i}^2 + \sum_{1 \leq i<j \leq N} 2w_i x_i w_j x_j \Big) \Big] \nonumber \\
&\rightarrow& \sum_{n=1}^W (\mu n^2 - \lambda) y_n^2 + \sum_{1 \leq n < m \leq W} 2(\lambda + \mu nm) y_n y_m  \nonumber \\
&& + \Big(\sum_{n=1}^W \sum_{i=1}^N (-2) \mu n w_i y_n x_i\Big) +  \Big(\sum_{i=1}^N \mu w_{i}^2 x_{i}^2 + \sum_{1 \leq i<j \leq N} 2 \mu w_i w_j x_i x_j \Big)   \nonumber \\

In [124]:
import numpy as np
import pandas as pd
import wildqat as wq

In [125]:
menu = pd.read_csv("~/Desktop/menu.csv", index_col=0)
menu.head()

,name,category,type,price,calorie,salt
id,,,,,,
1,彩りガーデンサラダ,sidedish,salad,299,130,1.1
2,小エビのサラダ,sidedish,salad,349,115,1.3
3,やわらかチキンのサラダ,sidedish,salad,299,134,1.2
4,わかめサラダ,sidedish,salad,299,92,2.1
5,イタリアンサラダ,sidedish,salad,299,196,0.7


In [126]:
class Item():
    def __init__(self, number, weight, cost):
        self.__number = number
        self.__name = name
        self.__weight = weight
        self.__cost = cost
    
    @property
    def name(self):
        return self.__name
    
    @property
    def weight(self):
        return self.__weight

    @property
    def cost(self):
        return self.__cost

    def __str__(self):
        return f"#{self.__number} (name : {self.name}, weight : {self.weight}, cost : {self.cost})"

In [127]:
WEIGHT = 1000

items = []
for _id, _detail in menu.iterrows():
    number = _id
    name = _detail["name"]
    weight = _detail["price"]
    cost = _detail["calorie"]
    items.append(Item(number=number, weight=weight, cost=cost))

In [128]:
def get_qubo(items, wlimit, A, B, C):
    # qubo行列を作成
    x_size = len(items)
    y_size = wlimit
    size = x_size + y_size
    
    # QUBO行列のx*x成分
    x_qubo = np.zeros((x_size, x_size), dtype=int)
    for i in range(x_size):
        wi = items[i].weight
        ci = items[i].cost
        for j in range(i, x_size):
            wj = items[j].weight
            if i == j:
                x_qubo[i][i] = A * wi*wi - B * ci
            else:
                x_qubo[i][j] = A * 2*wi*wj
    
    # QUBO行列のy*y成分
    y_qubo = np.zeros((y_size, y_size), dtype=int)
    for i in range(y_size):
        for j in range(i, y_size):
            if i == j:
                y_qubo[i][i] = A*(i+1)*(i+1) - C
            else:
                y_qubo[i][j] = 2*(C + A*(i+1)*(j+1))
                
    # QUBO行列のx*y成分
    xy_qubo = np.zeros((x_size, y_size), dtype=int)
    for i in range(x_size):
        wi = items[i].weight
        for j in range(y_size):
            xy_qubo[i][j] = -2*A*(j+1)*wi
    
    # QUBO行列のy*x成分
    yx_qubo = np.zeros((y_size, x_size), dtype=int)
    
    # QUBO行列を接合する
    upper_qubo = np.concatenate((x_qubo, xy_qubo), axis=1)
    lower_qubo = np.concatenate((yx_qubo, y_qubo), axis=1)
    qubo = np.concatenate((upper_qubo, lower_qubo), axis=0)
    
    return qubo


In [138]:
annealer = wq.opt()

In [167]:
annealer.qubo = get_qubo(items, wlimit=WEIGHT, A=1, B=300, C=5000000)
#: Descreasing rate of temperature [SA]
annealer.R = 0.999
#: Iterations [SA]
annealer.ite = 1000

In [ ]:
times = 1
for count in range(1, times+1):
    q = annealer.sa()
    
    answers = []
    weight = 0 
    cost = 0
    for i in range(len(items)):
        if q[i] > 0:
            answers.append(items[i])
            weight += items[i].weight
            cost += items[i].cost
    
    annealer.plot()
    
    print(f"count : {count}")
    print("y=1の要素数は {}.".format(sum(q[len(items):])))
    for answer in answers:
        print(f"selected : {answer}")
    print(f"total price : {weight}")
    print(f"total kcal : {cost}")
    print("")